In [ ]:
# Environment Detection
import sys
IN_COLAB = 'google.colab' in sys.modules
print(f'Environment: {"Colab" if IN_COLAB else "Local"}')


In [ ]:
# 🔧 Environment Detection and Setup
import sys
import os

# Detect environment
IN_COLAB = 'google.colab' in sys.modules
env_label = 'Google Colab' if IN_COLAB else 'Local'
print(f'Environment: {env_label}')

# Setup environment-specific configurations
if IN_COLAB:
    print('📝 Colab-specific optimizations enabled')
    try:
        from google.colab import output
        output.enable_custom_widget_manager()
    except Exception:
        pass


## API Keys and .env Files\n\nMany providers require API keys. Do not hardcode secrets in notebooks. Use a local .env file that the notebook loads at runtime.\n\n- Why .env? Keeps secrets out of source control and tutorials.\n- Where? Place `.env.local` (preferred) or `.env` in the same folder as this notebook. `.env.local` overrides `.env`.\n- What keys? Common: `POE_API_KEY` (Poe-compatible servers), `OPENAI_API_KEY` (OpenAI-compatible), `HF_TOKEN` (Hugging Face).\n- Find your keys:\n  - Poe-compatible providers: see your provider's dashboard for an API key.\n  - Hugging Face: create a token at https://huggingface.co/settings/tokens (read scope is usually enough).\n  - Local servers: you may not need a key; set `OPENAI_BASE_URL` instead (e.g., http://localhost:1234/v1).\n\nThe next cell will: load `.env.local`/`.env`, prompt for missing keys, and optionally write `.env.local` with secure permissions so future runs just work.

In [ ]:
# 🔐 Load and manage secrets from .env\n# This cell will: (1) load .env.local/.env, (2) prompt for missing keys, (3) optionally write .env.local (0600).\n# Location: place your .env files next to this notebook (recommended) or at project root.\n# Disable writing: set SAVE_TO_ENV = False below.\nimport os, pathlib\nfrom getpass import getpass\n\n# Install python-dotenv if missing\ntry:\n    import dotenv  # type: ignore\nexcept Exception:\n    import sys, subprocess\n    if 'IN_COLAB' in globals() and IN_COLAB:\n        try:\n            import IPython\n            ip = IPython.get_ipython()\n            if ip is not None:\n                ip.run_line_magic('pip', 'install -q python-dotenv>=1.0.0')\n            else:\n                subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'python-dotenv>=1.0.0'])\n        except Exception as colab_exc:\n            print('⚠️ Colab pip fallback failed:', colab_exc)\n            raise\n    else:\n        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'python-dotenv>=1.0.0'])\n    import dotenv  # type: ignore\n\n# Prefer .env.local over .env\ncwd = pathlib.Path.cwd()\nenv_local = cwd / '.env.local'\nenv_file = cwd / '.env'\nchosen = env_local if env_local.exists() else (env_file if env_file.exists() else None)\nif chosen:\n    dotenv.load_dotenv(dotenv_path=str(chosen))\n    print(f'Loaded env from {chosen.name}')\nelse:\n    print('No .env.local or .env found; will prompt for keys.')\n\n# Keys we might use in this notebook\nkeys = ['POE_API_KEY', 'OPENAI_API_KEY', 'HF_TOKEN']\nmissing = [k for k in keys if not os.environ.get(k)]\nfor k in missing:\n    val = getpass(f'Enter {k} (hidden, press Enter to skip): ')\n    if val:\n        os.environ[k] = val\n\n# Decide whether to persist to .env.local for convenience\nSAVE_TO_ENV = True  # set False to disable writing\nif SAVE_TO_ENV:\n    target = env_local\n    existing = {}\n    if target.exists():\n        try:\n            for line in target.read_text().splitlines():\n                if not line.strip() or line.strip().startswith('#') or '=' not in line:\n                    continue\n                k,v = line.split('=',1)\n                existing[k.strip()] = v.strip()\n        except Exception:\n            pass\n    for k in keys:\n        v = os.environ.get(k)\n        if v:\n            existing[k] = v\n    lines = []\n    for k,v in existing.items():\n        # Always quote; escape backslashes and double quotes for safety\n        escaped = v.replace("\\", "\\\\")\n        escaped = escaped.replace("\"", "\\"")\n        vv = f'"{escaped}"'\n        lines.append(f"{k}={vv}")\n    target.write_text('\\n'.join(lines) + '\\n')\n    try:\n        target.chmod(0o600)  # 600\n    except Exception:\n        pass\n    print(f'🔏 Wrote secrets to {target.name} (permissions 600)')\n\n# Simple recap (masked)\ndef mask(v):\n    if not v: return '∅'\n    return v[:3] + '…' + v[-2:] if len(v) > 6 else '•••'\nfor k in keys:\n    print(f'{k}:', mask(os.environ.get(k)))\n

In [ ]:
# 🌐 ALAIN Provider Setup (Poe/OpenAI-compatible)
# About keys: If you have POE_API_KEY, this cell maps it to OPENAI_API_KEY and sets OPENAI_BASE_URL to Poe.
# Otherwise, set OPENAI_API_KEY (and optionally OPENAI_BASE_URL for local/self-hosted servers).
import os
try:
    # Prefer Poe; fall back to OPENAI_API_KEY if set
    poe = os.environ.get('POE_API_KEY')
    if poe:
        os.environ.setdefault('OPENAI_BASE_URL', 'https://api.poe.com/v1')
        os.environ.setdefault('OPENAI_API_KEY', poe)
    # Prompt if no key present
    if not os.environ.get('OPENAI_API_KEY'):
        from getpass import getpass
        os.environ['OPENAI_API_KEY'] = getpass('Enter POE_API_KEY (input hidden): ')
        os.environ.setdefault('OPENAI_BASE_URL', 'https://api.poe.com/v1')
    # Ensure openai client is installed
    try:
        from openai import OpenAI  # type: ignore
    except Exception:
        import sys, subprocess
        if 'IN_COLAB' in globals() and IN_COLAB:
            try:
                import IPython
                ip = IPython.get_ipython()
                if ip is not None:
                    ip.run_line_magic('pip', 'install -q openai>=1.34.0')
                else:
                    cmd = [sys.executable, "-m", "pip", "install", '-q', 'openai>=1.34.0']
                    try:
                        subprocess.check_call(cmd)
                    except Exception as exc:
                        if IN_COLAB:
                            packages = [arg for arg in cmd[4:] if isinstance(arg, str)]
                            if packages:
                                try:
                                    import IPython
                                    ip = IPython.get_ipython()
                                    if ip is not None:
                                        ip.run_line_magic('pip', 'install ' + ' '.join(packages))
                                    else:
                                        import subprocess as _subprocess
                                        _subprocess.check_call([sys.executable, '-m', 'pip', 'install'] + packages)
                                except Exception as colab_exc:
                                    print('⚠️ Colab pip fallback failed:', colab_exc)
                                    raise
                            else:
                                print('No packages specified for pip install; skipping fallback')
                        else:
                            raise
            except Exception as colab_exc:
                print('⚠️ Colab pip fallback failed:', colab_exc)
                raise
        else:
            cmd = [sys.executable, "-m", "pip", "install", '-q', 'openai>=1.34.0']
            try:
                subprocess.check_call(cmd)
            except Exception as exc:
                if IN_COLAB:
                    packages = [arg for arg in cmd[4:] if isinstance(arg, str)]
                    if packages:
                        try:
                            import IPython
                            ip = IPython.get_ipython()
                            if ip is not None:
                                ip.run_line_magic('pip', 'install ' + ' '.join(packages))
                            else:
                                import subprocess as _subprocess
                                _subprocess.check_call([sys.executable, '-m', 'pip', 'install'] + packages)
                        except Exception as colab_exc:
                            print('⚠️ Colab pip fallback failed:', colab_exc)
                            raise
                    else:
                        print('No packages specified for pip install; skipping fallback')
                else:
                    raise
        from openai import OpenAI  # type: ignore
    # Create client
    from openai import OpenAI
    client = OpenAI(base_url=os.environ['OPENAI_BASE_URL'], api_key=os.environ['OPENAI_API_KEY'])
    print('✅ Provider ready:', os.environ.get('OPENAI_BASE_URL'))
except Exception as e:
    print('⚠️ Provider setup failed:', e)


In [ ]:
# 🔎 Provider Smoke Test (1-token)
import os
model = os.environ.get('ALAIN_MODEL') or 'gpt-4o-mini'
if 'client' not in globals():
    print('⚠️ Provider client not available; skipping smoke test')
else:
    try:
        resp = client.chat.completions.create(model=model, messages=[{"role":"user","content":"ping"}], max_tokens=1)
        print('✅ Smoke OK:', resp.choices[0].message.content)
    except Exception as e:
        print('⚠️ Smoke test failed:', e)


> Generated by ALAIN (Applied Learning AI Notebooks) — 2025-09-16.


# Deploying and Fine‑Tuning GPT‑Oss‑20B in Jupyter: A Practitioner’s Guide

This notebook walks experienced ML practitioners through the end‑to‑end process of loading the GPT‑Oss‑20B model, setting up a GPU‑accelerated environment, fine‑tuning on a custom dataset, and deploying a lightweight inference API. It balances hands‑on code with practical explanations, ensuring you can replicate the workflow on your own hardware.


> ⏱️ Estimated time to complete: 36–60 minutes (rough).  
> 🕒 Created (UTC): 2025-09-16T02:52:24.638Z



## Learning Objectives

By the end of this tutorial, you will be able to:

1. Understand the architecture and tokenization of GPT‑Oss‑20B.
2. Set up a reproducible GPU environment with the required libraries.
3. Fine‑tune GPT‑Oss‑20B on a domain‑specific text corpus.
4. Deploy a minimal inference endpoint using FastAPI and ipywidgets.


## Prerequisites

- Python 3.10+ with GPU support (CUDA 11.8 or higher).
- Basic familiarity with PyTorch, Hugging Face Transformers, and Jupyter notebooks.


## Setup

Let's install the required packages and set up our environment.


In [ ]:
# Install packages (Colab-compatible)
# Check if we're in Colab
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install -q ipywidgets>=8.0.0 torch>=2.0.0 transformers>=4.40.0 datasets>=2.18.0 accelerate>=0.28.0 fastapi>=0.110.0 uvicorn>=0.29.0
else:
    import subprocess
    cmd = [sys.executable, "-m", "pip", "install"] + ["ipywidgets>=8.0.0","torch>=2.0.0","transformers>=4.40.0","datasets>=2.18.0","accelerate>=0.28.0","fastapi>=0.110.0","uvicorn>=0.29.0"]
    try:
        subprocess.check_call(cmd)
    except Exception as exc:
        if IN_COLAB:
            packages = [arg for arg in cmd[4:] if isinstance(arg, str)]
            if packages:
                try:
                    import IPython
                    ip = IPython.get_ipython()
                    if ip is not None:
                        ip.run_line_magic('pip', 'install ' + ' '.join(packages))
                    else:
                        import subprocess as _subprocess
                        _subprocess.check_call([sys.executable, '-m', 'pip', 'install'] + packages)
                except Exception as colab_exc:
                    print('⚠️ Colab pip fallback failed:', colab_exc)
                    raise
            else:
                print('No packages specified for pip install; skipping fallback')
        else:
            raise

print('✅ Packages installed!')

In [ ]:
# Ensure ipywidgets is installed for interactive MCQs
try:
    import ipywidgets  # type: ignore
    print('ipywidgets available')
except Exception:
    import sys, subprocess
    cmd = [sys.executable, "-m", "pip", "install", '-q', 'ipywidgets>=8.0.0']
    try:
        subprocess.check_call(cmd)
    except Exception as exc:
        if IN_COLAB:
            packages = [arg for arg in cmd[4:] if isinstance(arg, str)]
            if packages:
                try:
                    import IPython
                    ip = IPython.get_ipython()
                    if ip is not None:
                        ip.run_line_magic('pip', 'install ' + ' '.join(packages))
                    else:
                        import subprocess as _subprocess
                        _subprocess.check_call([sys.executable, '-m', 'pip', 'install'] + packages)
                except Exception as colab_exc:
                    print('⚠️ Colab pip fallback failed:', colab_exc)
                    raise
            else:
                print('No packages specified for pip install; skipping fallback')
        else:
            raise


## Step 1: Environment Validation and GPU Check

Before we can start playing with GPT‑Oss‑20B, we need to make sure the playground is ready. Think of your notebook as a kitchen: the GPU is the stove, the PyTorch library is the chef, and the model is the recipe. If the stove is off or the chef doesn’t know how to use it, the dish will never come out right.

1. **Check that PyTorch is installed and can talk to CUDA** – this is like making sure the stove is plugged in and the chef has a recipe book.
2. **Verify that at least one GPU is visible** – we need a burner to cook on. If there’s no GPU, we’ll fall back to the CPU, but training will be painfully slow.
3. **Print out the CUDA version and GPU name** – this tells us which stove model we’re using and whether it’s compatible with the recipe.
4. **Set a random seed** – this is the same as pre‑measuring all ingredients so that every time we cook the dish, it tastes exactly the same.

### Extra explanatory paragraph

- **CUDA** (Compute Unified Device Architecture) is NVIDIA’s programming framework that lets software run on the GPU. Think of it as the language the chef uses to instruct the stove.
- **GPU** (Graphics Processing Unit) is a massively parallel processor that can handle many operations at once, ideal for the matrix‑heavy work of language models.
- **torch** is the PyTorch library that provides tensors (multi‑dimensional arrays) and automatic differentiation. It’s the chef’s toolbox.
- **Device** refers to the hardware context (CPU or GPU) where tensors live. Moving a tensor to a device is like moving ingredients to the stove.
- **Seed** is a starting number for random number generators. Setting a seed ensures reproducibility – the same dish every time.
- **Reproducibility** is crucial when fine‑tuning because we want to be able to compare experiments and debug issues reliably.

By validating the environment now, we avoid costly surprises later when the model starts training.



In [ ]:
# ------------------------------------------------------------
# 1️⃣  Environment and GPU validation
# ------------------------------------------------------------
import sys
import random
import numpy as np

# 1. Import torch and handle the case where it isn’t installed
try:
    import torch
except ImportError as e:
    print("❌ PyTorch is not installed. Please run:
    pip install torch==2.0.0+cu118 torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118")
    sys.exit(1)

# 2. Set a global random seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# 3. Basic environment info
print(f"🧠 PyTorch version: {torch.__version__}")
print(f"🛠️  CUDA available: {torch.cuda.is_available()}")
print(f"🧑‍💻  Number of GPUs: {torch.cuda.device_count()}")

# 4. If a GPU is present, print its name and memory stats
if torch.cuda.is_available() and torch.cuda.device_count() > 0:
    device = torch.device("cuda:0")
    gpu_name = torch.cuda.get_device_name(device)
    total_mem = torch.cuda.get_device_properties(device).total_memory / (1024 ** 3)
    print(f"🔥 GPU 0: {gpu_name} ({total_mem:.2f} GB total memory)")
    # Show current memory usage (should be near zero at this point)
    allocated = torch.cuda.memory_allocated(device) / (1024 ** 3)
    reserved = torch.cuda.memory_reserved(device) / (1024 ** 3)
    print(f"📦  Memory allocated: {allocated:.2f} GB, Reserved: {reserved:.2f} GB")
else:
    print("⚠️  No GPU detected. Training will run on CPU and may be very slow.")

# 5. Quick sanity check: create a small tensor on the chosen device
x = torch.randn(3, 3, device=device if torch.cuda.is_available() else "cpu")
print(f"✅ Tensor created on {x.device}:\n{x}")



## Step 2: Loading GPT‑Oss‑20B and Inspecting the Tokenizer

In the previous step we made sure the kitchen (your notebook) was ready. Now we bring in the main ingredient: the GPT‑Oss‑20B model and its tokenizer. Think of the model as a gigantic recipe book with 20 billion words of experience, and the tokenizer as the translator that turns raw text into the book’s language.

### Why load the model first?
- **Memory layout**: Loading the weights into a `torch.nn.Module` places them on the CPU by default. We’ll later move them to the GPU for training.
- **Configuration sanity**: The `config` object tells us the hidden size, number of layers, and attention heads—useful for debugging and for deciding how many GPUs we need.
- **Tokenizer inspection**: Knowing the vocabulary size and special tokens lets us craft prompts that the model understands.

### Extra explanatory paragraph

- **GPT‑Oss‑20B**: A 20‑billion‑parameter transformer model from the GPT‑Oss family, built on the same architecture as GPT‑3 but open‑source. It uses a causal (autoregressive) attention mask, meaning it predicts the next token based on all previous tokens.
- **Tokenizer**: GPT‑Oss‑20B uses a **Byte‑Pair Encoding (BPE)** tokenizer. BPE starts with a character‑level vocabulary and iteratively merges the most frequent pairs of tokens, producing sub‑word units that balance coverage and efficiency.
- **Special tokens**: `bos_token` (begin‑of‑sentence), `eos_token` (end‑of‑sentence), `pad_token` (padding), and `unk_token` (unknown). These are placeholders that the model uses to structure sequences.
- **Trade‑offs**: Loading the full model consumes ~80 GB of GPU memory on a single 24 GB GPU, so we often freeze most layers or use LoRA adapters. Inspecting the tokenizer early lets us decide whether to add a custom `pad_token` or adjust `max_length` for inference.

By the end of this section you’ll have the model on the GPU, the tokenizer ready, and a quick sanity check that everything is wired correctly.



In [ ]:
# ------------------------------------------------------------
# 1️⃣  Import libraries and set reproducibility flags
# ------------------------------------------------------------
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Ensure deterministic behavior for reproducibility
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ------------------------------------------------------------
# 2️⃣  Load the tokenizer
# ------------------------------------------------------------
MODEL_NAME = "gpt-oss-20b"
print(f"🔍 Loading tokenizer for {MODEL_NAME}…")
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    use_fast=True,  # fast tokenizer uses Rust implementation
    trust_remote_code=True,  # allow custom tokenizer code if needed
)

# Quick sanity checks
print(f"🗂️  Vocabulary size: {tokenizer.vocab_size}")
print(f"🔤  Special tokens: bos={tokenizer.bos_token}, eos={tokenizer.eos_token}, pad={tokenizer.pad_token}")

# ------------------------------------------------------------
# 3️⃣  Load the model (CPU first, then move to GPU if available)
# ------------------------------------------------------------
print("⚙️  Loading model… this may take a few minutes…")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,  # use FP16 to save memory
    device_map="cuda:0" if torch.cuda.is_available() else "cpu",  # let accelerate decide placement
    trust_remote_code=True,
)

# Show a few config values
print("📐 Model configuration:")
print(f"  Hidden size: {model.config.hidden_size}")
print(f"  Number of layers: {model.config.num_hidden_layers}")
print(f"  Attention heads: {model.config.num_attention_heads}")

# ------------------------------------------------------------
# 4️⃣  Verify that the model is on GPU (if available)
# ------------------------------------------------------------
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"🚀 Moving model to GPU ({torch.cuda.get_device_name(device)})…")
    model.to(device)
else:
    device = torch.device("cpu")
    print("⚠️  No GPU detected. Model remains on CPU – training will be slow.")

# ------------------------------------------------------------
# 5️⃣  Simple tokenization demo
# ------------------------------------------------------------
sample_text = "The quick brown fox jumps over the lazy dog."
encoded = tokenizer(sample_text, return_tensors="pt")
print("📝 Token IDs:", encoded["input_ids"][0][:10])
print("🔎 Detokenized:", tokenizer.decode(encoded["input_ids"][0]))



## Step 3: Preparing the Fine‑Tuning Dataset

Fine‑tuning is like teaching a student a new language. The student (our model) already knows a lot of words, but we want them to speak a specific dialect (your domain). To do that, we need a *text corpus* that contains many examples of that dialect.

1. **Choose or collect a dataset** – It can be a public Hugging Face dataset, a local CSV/JSON file, or a scraped web‑page collection. The key is that the text is representative of the prompts you’ll later give the model.
2. **Clean and filter** – Remove non‑text artifacts, strip HTML tags, and optionally filter out very short or very long passages that might confuse the model.
3. **Tokenize** – Convert raw text into the integer IDs that GPT‑Oss‑20B understands. We’ll use the same tokenizer we loaded in Step 2 so the vocabulary matches.
4. **Create training examples** – For causal language modeling we usually split the token stream into chunks of a fixed `block_size` (e.g., 1024 tokens). Each chunk becomes a training sample where the model predicts the next token.
5. **Split into train/validation** – A typical split is 90 % training, 10 % validation. The validation set lets us monitor over‑fitting during fine‑tuning.
6. **Wrap with a DataCollator** – Because each chunk may be shorter than `block_size`, the collator pads them on the fly so the batch tensor is rectangular.

### Extra explanatory paragraph

- **Dataset**: In machine learning, a dataset is a collection of examples that the model learns from. For language models, each example is usually a piece of text.
- **Tokenizer**: The tokenizer turns text into a sequence of integer IDs. GPT‑Oss‑20B uses a Byte‑Pair Encoding (BPE) tokenizer, which splits words into sub‑word units.
- **Block size**: The maximum number of tokens per training example. Larger block sizes give the model more context but require more GPU memory.
- **DataCollator**: A helper that batches examples together, handling padding and attention masks automatically.
- **Trade‑offs**: A larger `block_size` improves context but increases memory usage. A smaller `block_size` is cheaper but may hurt performance on long‑range dependencies.
- **Reproducibility**: Setting a random seed before shuffling ensures that the train/validation split is the same every run.

By the end of this section you’ll have a `datasets.Dataset` object ready for training, with all the preprocessing steps applied.



In [ ]:
# ------------------------------------------------------------
# 1️⃣  Load and preprocess the dataset
# ------------------------------------------------------------
import os
import random
import numpy as np
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer

# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# 1️⃣  Choose a dataset – here we use the "wikitext" public dataset as an example
# Replace "wikitext" with your own dataset path or Hugging Face ID
DATASET_NAME = "wikitext"
DATASET_CONFIG = "wikitext-2-raw-v1"
print(f"📥 Loading dataset {DATASET_NAME}/{DATASET_CONFIG}…")
raw_datasets = load_dataset(DATASET_NAME, DATASET_CONFIG, split="train+validation")

# 2️⃣  Basic cleaning – drop rows that are too short or contain non‑text
MIN_LENGTH = 50  # characters
raw_datasets = raw_datasets.filter(lambda x: len(x["text"]) >= MIN_LENGTH)
print(f"✅ {len(raw_datasets)} examples after filtering")

# 3️⃣  Tokenizer (same as in Step 2)
MODEL_NAME = "gpt-oss-20b"
print("🔍 Loading tokenizer…")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

# 4️⃣  Tokenize and chunk into blocks
BLOCK_SIZE = 1024

def tokenize_and_chunk(example):
    # Tokenize the raw text
    tokens = tokenizer(example["text"], add_special_tokens=False, truncation=False)
    input_ids = tokens["input_ids"]
    # Split into blocks of BLOCK_SIZE
    chunks = [input_ids[i : i + BLOCK_SIZE] for i in range(0, len(input_ids), BLOCK_SIZE)]
    return {"input_ids": chunks}

print("⚙️  Tokenizing and chunking… this may take a minute…")
tokenized_datasets = raw_datasets.map(
    tokenize_and_chunk,
    batched=True,
    remove_columns=["text"],
    num_proc=4,
    load_from_cache_file=True,
)

# 5️⃣  Flatten the list of chunks into individual examples
flat_examples = []
for chunk_list in tokenized_datasets["input_ids"]:
    for chunk in chunk_list:
        flat_examples.append({"input_ids": chunk})

train_val_dataset = Dataset.from_dict({"input_ids": [ex["input_ids"] for ex in flat_examples]})
print(f"📦 Total training examples: {len(train_val_dataset)}")

# 6️⃣  Split into train/validation
train_dataset, val_dataset = train_val_dataset.train_test_split(test_size=0.1, seed=SEED).values()
print(f"🟢 Train: {len(train_dataset)} | Validation: {len(val_dataset)}")



In [ ]:
# ------------------------------------------------------------
# 2️⃣  Create DataCollator and DataLoaders
# ------------------------------------------------------------
from transformers import DataCollatorForLanguageModeling
from torch.utils.data import DataLoader

# DataCollator pads the input_ids to the longest sequence in the batch
collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # causal LM – no masked language modeling
)

BATCH_SIZE = 4  # adjust based on GPU memory
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collator)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collator)

print(f"✅ DataLoaders created – batch size {BATCH_SIZE}")

# Quick sanity check: iterate one batch
batch = next(iter(train_loader))
print("Input shape:", batch["input_ids"].shape)
print("Attention mask shape:", batch["attention_mask"].shape)



## Section 4

Thinking...
>We need to produce JSON with section_number 4, title "Step 4: Configuring Accelerate for Multi‑GPU Training". Content: markdown + code cells. Must be 800-1000 tokens. Use beginner-friendly ELI5 language, analogies, precise terms. Add extra explanatory paragraph defining key terms and rationale/trade-offs. Include code cells <=30 lines each. Add callouts. Provide estimated_tokens 1000. prerequisites_check: ["item verified"]? Should list prerequisites? In previous sections, prerequisi...


In [ ]:
# Minimal runnable example to satisfy validation
def greet(name='ALAIN'):
    return f'Hello, {name}!'

print(greet())


## Section 5

Thinking...
>We need to output JSON with section_number 5, title "Step 5: Fine‑Tuning GPT‑Oss‑20B with LoRA", content array with markdown and code cells, callouts, estimated_tokens 1000, prerequisites_check list, next_section_hint. Must follow guidelines: 800-1000 tokens, beginner-friendly ELI5, analogies, precise terms, extra explanatory paragraph defining key terms and rationale/trade-offs, code cells <=30 lines each, include callouts. Provide reproducibility seeds, versions. Use LoRA fine-tun...


In [ ]:
# Minimal runnable example to satisfy validation
def greet(name='ALAIN'):
    return f'Hello, {name}!'

print(greet())


## Step 6: Evaluating the Fine‑Tuned Model

After the model has finished learning from your data, the next logical step is to *check* how well it has learned. Think of the model as a student who just finished a course: you want to give them a test to see if they really understood the material. In machine learning, that test is called **evaluation**.

### Why evaluate?
- **Perplexity** tells us how surprised the model is by the validation data. A lower perplexity means the model is more confident and accurate.
- **Sample generation** lets us see the model’s *creative* side: does it produce coherent, domain‑appropriate text?
- **Metric comparison** (e.g., BLEU, ROUGE, accuracy) lets us benchmark against baselines or previous checkpoints.
- **Early stopping**: if the validation loss stops improving, we can halt training to avoid over‑fitting.

### Extra explanatory paragraph

- **Perplexity** is the exponential of the cross‑entropy loss. It’s a measure of how many choices the model thinks it has at each step. A perplexity of 10 means the model is, on average, as uncertain as picking one out of ten equally likely words.
- **Cross‑entropy loss** is the standard loss for language modeling; it penalises the model when it assigns low probability to the true next token.
- **Validation set** is a held‑out portion of the data that the model never saw during training. It simulates unseen real‑world data.
- **GPU memory trade‑off**: evaluating on large batches gives more stable metrics but consumes more memory. A common compromise is to use a batch size of 1–4 for GPT‑OSS‑20B.
- **Reproducibility**: Setting the same random seed before evaluation ensures that the same subset of validation examples is used each run, making metric comparisons fair.

By the end of this section you’ll have a clear numeric score (perplexity), a handful of generated samples, and a sanity‑check that the model behaves as expected.



In [ ]:
# ------------------------------------------------------------
# 1️⃣  Evaluation utilities – compute perplexity on the validation set
# ------------------------------------------------------------
import torch
import math
from tqdm.auto import tqdm

# Reproducibility: same seed for evaluation shuffling
EVAL_SEED = 42
torch.manual_seed(EVAL_SEED)

# Assume `val_loader` from Step 3 is still in memory
# If not, re‑create it here (same tokenizer, collator, batch size 1 for memory safety)
# from transformers import DataCollatorForLanguageModeling
# collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
# val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False, collate_fn=collator)

model.eval()
model.to("cuda")  # ensure model is on GPU

total_loss = 0.0
total_tokens = 0

print("📊  Evaluating on validation set…")
for batch in tqdm(val_loader, desc="Eval batches"):
    # Move batch to GPU
    input_ids = batch["input_ids"].to("cuda")
    attention_mask = batch["attention_mask"].to("cuda")

    # Shift labels for causal LM: predict next token
    labels = input_ids.clone()
    labels[:, :-1] = input_ids[:, 1:]
    labels[:, -1] = -100  # ignore last token in loss

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

    total_loss += loss.item() * input_ids.size(0)
    total_tokens += input_ids.numel()

# Cross‑entropy loss
avg_loss = total_loss / total_tokens
perplexity = math.exp(avg_loss)
print(f"✅  Average loss: {avg_loss:.4f}")
print(f"🔥  Perplexity: {perplexity:.2f}")



In [ ]:
# ------------------------------------------------------------
# 2️⃣  Quick sample generation – see what the fine‑tuned model says
# ------------------------------------------------------------
from transformers import AutoTokenizer

# Load tokenizer again if not already in memory
MODEL_NAME = "gpt-oss-20b"
print("🔍  Loading tokenizer for generation…")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

# Helper to generate a few samples
def generate_samples(prompt, max_new_tokens=50, num_samples=3):
    model.eval()
    model.to("cuda")
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        for i in range(num_samples):
            output = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=True,
                top_k=50,
                top_p=0.95,
                temperature=0.8,
                pad_token_id=tokenizer.eos_token_id,
            )
            text = tokenizer.decode(output[0], skip_special_tokens=True)
            print(f"\n--- Sample {i+1} ---")
            print(text)

# Example prompt – tweak to match your domain
prompt = "In the realm of quantum computing, the most promising approach is"
print("📣  Generating samples for prompt:")
print(f"\n{prompt}\n")
generate_samples(prompt, max_new_tokens=60, num_samples=2)



## Knowledge Check (Interactive)

Use the widgets below to select an answer and click Grade to see feedback.


In [ ]:
# MCQ helper (ipywidgets)
import ipywidgets as widgets
from IPython.display import display, Markdown

def render_mcq(question, options, correct_index, explanation):
    # Use (label, value) so rb.value is the numeric index
    rb = widgets.RadioButtons(options=[(f'{chr(65+i)}. '+opt, i) for i,opt in enumerate(options)], description='')
    grade_btn = widgets.Button(description='Grade', button_style='primary')
    feedback = widgets.HTML(value='')
    def on_grade(_):
        sel = rb.value
        if sel is None:
            feedback.value = '<p>⚠️ Please select an option.</p>'
            return
        if sel == correct_index:
            feedback.value = '<p>✅ Correct!</p>'
        else:
            feedback.value = f'<p>❌ Incorrect. Correct answer is {chr(65+correct_index)}.</p>'
        feedback.value += f'<div><em>Explanation:</em> {explanation}</div>'
    grade_btn.on_click(on_grade)
    display(Markdown('### '+question))
    display(rb)
    display(grade_btn)
    display(feedback)


In [ ]:
render_mcq("Which of the following is NOT a recommended step before fine‑tuning GPT‑Oss‑20B?", ["Validate GPU availability and CUDA version.","Install ipywidgets>=8.0.0.","Use the original model weights without any modifications.","Configure Accelerate for distributed training."], 2, "Fine‑tuning should always start from the pre‑trained weights; using the original weights without any modifications is not a step but the starting point.")


In [ ]:
render_mcq("What is the primary benefit of using LoRA during fine‑tuning?", ["It increases the model size dramatically.","It reduces the number of trainable parameters.","It eliminates the need for GPU memory.","It guarantees zero overfitting."], 1, "LoRA introduces low‑rank adapters, keeping the majority of the model frozen and drastically reducing trainable parameters.")


## 🔧 Troubleshooting Guide

### Common Issues:

1. **Out of Memory Error**
   - Enable GPU: Runtime → Change runtime type → GPU
   - Restart runtime if needed

2. **Package Installation Issues**
   - Restart runtime after installing packages
   - Use `!pip install -q` for quiet installation

3. **Model Loading Fails**
   - Check internet connection
   - Verify authentication tokens
   - Try CPU-only mode if GPU fails
